## 1. load evaluation results

In [2]:
import pandas as pd 
import numpy as np

pp_metric = pd.read_json('./pp/metrics.json')
pp_preds = pd.read_json('./pp/predictions.json')

blendsql_metric = pd.read_json('./gpt-4-blendsql-plus-pp/metrics.json')
blendsql_preds = pd.read_json('./gpt-4-blendsql-plus-pp/predictions.json')

In [3]:
agg_table = pd.concat( [blendsql_preds[['idx', 'question', 'answer_text', 'pred_sql', 'pred_text']], pp_preds[[ 'pred_text']]], axis = 1)
agg_table.columns = ['idx','question', 'answer_text','pred_sql', 'blendsql_text', 'gpt_text']
agg_table

,idx,question,answer_text,pred_sql,blendsql_text,gpt_text
0,0,What is the middle name of the player with the...,Jerry,{{\n LLMQA(\n 'What is the middle na...,Jerry,
1,1,What was the nickname of the gold medal winner...,Starke Rudolf,{{\n LLMQA(\n 'What was the nickname...,Johan Rudolf Starke,Rudolf Starke
2,2,The driver who finished in position 4 in the 2...,British,{{\n LLMQA(\n 'What nationality is t...,British,British
3,3,What is that address of the museum located in ...,503 Peeples Street SW,{{\n LLMQA(\n 'What is the address o...,The address of the museum located in a Victori...,The address of the museum located in a Victori...
4,4,What place was achieved by the person who fini...,sixth,"SELECT ""position"" FROM w WHERE ""time"" = '2:13....",6th,6th
...,...,...,...,...,...,...
3461,3461,What is the league whose city has a history of...,Hong Kong Premier League,"SELECT ""league"" FROM w WHERE ""city"" = {{\n ...",Chinese Super League,Chinese Super League
3462,3462,When was the album by the artist born in Redla...,"April 4 , 1988","SELECT {{LLMMap('Release date of the album?', ...","The album by the artist born in Redlands, Cali...","The album by the artist born in Redlands, Cali..."
3463,3463,Of the free and open source software browsers ...,Android,{{\n LLMQA(\n 'Which free and open s...,Android (operating system),The table does not provide information on whic...
3464,3464,In the year the Pro Tour event took place in P...,17 August 1997,{{\n LLMQA(\n 'When did the World Ch...,1997-8-17,1997-8-17


## 2. observation level metrics

In [10]:
import datasets
import sys

sys.path.insert(0, '../')
metric = datasets.load.load_metric(path = '../research/metrics/hybridqa/')

/Users/caesa/miniconda3/envs/blendsql/lib/python3.9/site-packages/datasets/load.py:848: FutureWarning: The repository for hybridqa contains custom code which must be executed to correctly load the metric. You can inspect the repository content at ../research/metrics/hybridqa/hybridqa.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [11]:
for model in ['gpt_', 'blendsql_']:

    metric_column = []
    for test_idx in range(3466):
        metric_dict = dict()
        metric_dict['predictions'] = agg_table[model + 'text'].values[test_idx]
        metric_dict['reference'] = {'answer_text': agg_table['answer_text'].values[test_idx], 
                                    'id': agg_table['idx'].values[test_idx],
                                    'question': agg_table['question'].values[test_idx]}
    
        metric.add(**metric_dict)
        metric_detail = metric.compute()
        metric_column.append(metric_detail)        

    model_metric = pd.DataFrame(metric_column)
    model_metric.columns = [model + i for i in model_metric.columns]
    agg_table = pd.concat([agg_table, model_metric], axis = 1)
    


In [13]:
print(agg_table[['gpt_denotation_acc','blendsql_denotation_acc']].mean().values)

[0.49134449 0.57761108]


## 3. analyze the result

In [14]:
show_metric = 'denotation_acc'
show_columns = ['idx', 'question', 'answer_text', 'pred_sql', 'blendsql_text', 'gpt_text', 'blendsql_' + show_metric, 'gpt_'+ show_metric,]

### 3.1 the cases when blendsql outperforms gpt, what is the pattern?

In [15]:
win_flag = agg_table['gpt_'+ show_metric] < agg_table['blendsql_' + show_metric]
blendsql_win = agg_table[win_flag][show_columns]
blendsql_win

,idx,question,answer_text,pred_sql,blendsql_text,gpt_text,blendsql_denotation_acc,gpt_denotation_acc
0,0,What is the middle name of the player with the...,Jerry,{{\n LLMQA(\n 'What is the middle na...,Jerry,,1.0,0.0
14,14,Who was the husband of the mother of Ottoman s...,Selim I,{{\n LLMQA(\n 'Who was the husband o...,Selim I,,1.0,0.0
20,20,Which team of the Cornwall League 1 comes from...,Veor,"SELECT ""team"" FROM w WHERE ""town/village"" = {{...",veor,Camborne School of Mines,1.0,0.0
22,22,What year does Orson Scott Card 's 1986 novel ...,5270,{{\n LLMQA(\n 'What year does the no...,5270,"The novel ""Speaker for the Dead"" takes place a...",1.0,0.0
23,23,How many people performed on the most recent s...,five,{{\n LLMQA(\n 'How many people perfo...,5,1,1.0,0.0
...,...,...,...,...,...,...,...,...
3442,3442,"A solo-artist born in Dawson , Georgia who qui...",Shane Brennan,{{\n LLMQA(\n 'Who was the new showr...,Shane Brennan,"Otis Redding, Shane Brennan",1.0,0.0
3455,3455,How old was the discover of the Aglycone for t...,71,{{\n LLMQA(\n 'How old was the disco...,71,"Josef Herzig, the discoverer of the structure ...",1.0,0.0
3456,3456,What kind of journeying show did the father of...,medicine show,{{\n LLMQA(\n 'What kind of journeyi...,Medicine show,His father had a medicine show.,1.0,0.0
3460,3460,How many Sikh gurdwaras are in the Indian city...,three,{{\n LLMQA(\n 'How many Sikh gurdwar...,3,The text does not provide information on the n...,1.0,0.0


In [17]:
blendsql_win.to_csv('./blendsql_win.csv',index = False)

The answer provided by blensql is more concise and more closedly related to the table via the SQL-like syntax. Hence, as it is compared to answers of open eneded questions, it tends to perform better for questions expecting more regulated contents from the heterogeneous table. For exampale the quesitons that are expecting a concise numerical answer from the table.

### 3.2 when a mistake is made, do blendsql and gpt make the same mistakes?

In [18]:
mistake_flag = (agg_table['gpt_'+ show_metric] == 0.0) | (agg_table['blendsql_'+ show_metric] == 0.0)
agg_mistake = agg_table[mistake_flag][show_columns]
print("""Judged by non-unitary %s, there are in total %d mistakes out of %d total observation, which amount to a pct mistake rate of %.2f """ % 
      (show_metric, agg_mistake.shape[0], agg_table.shape[0], agg_mistake.shape[0] / agg_table.shape[0]))

common_flag = agg_mistake['blendsql_'+show_metric] ==  agg_mistake['gpt_'+show_metric]
common_mistake = agg_mistake[common_flag]

print("""Judged by non-unitary %s, there are in total %d common mistakes out of %d total mistakes, which amounts to a pct common mistake of %.2f"""% 
      (show_metric, common_mistake.shape[0], agg_mistake.shape[0], common_mistake.shape[0] / agg_mistake.shape[0]))

Judged by non-unitary denotation_acc, there are in total 1962 mistakes out of 3466 total observation, which amount to a pct mistake rate of 0.57 
Judged by non-unitary denotation_acc, there are in total 1265 common mistakes out of 1962 total mistakes, which amounts to a pct common mistake of 0.64


In [19]:
common_mistake

,idx,question,answer_text,pred_sql,blendsql_text,gpt_text,blendsql_denotation_acc,gpt_denotation_acc
1,1,What was the nickname of the gold medal winner...,Starke Rudolf,{{\n LLMQA(\n 'What was the nickname...,Johan Rudolf Starke,Rudolf Starke,0.0,0.0
3,3,What is that address of the museum located in ...,503 Peeples Street SW,{{\n LLMQA(\n 'What is the address o...,The address of the museum located in a Victori...,The address of the museum located in a Victori...,0.0,0.0
4,4,What place was achieved by the person who fini...,sixth,"SELECT ""position"" FROM w WHERE ""time"" = '2:13....",6th,6th,0.0,0.0
6,6,How many years was the Mexican billionaire Sli...,3,{{\n LLMQA(\n 'How many years was Sl...,4,4 years,0.0,0.0
7,7,What is the difference in time between José Re...,0:06,{{\n LLMQA(\n 'What is the time diff...,0:17,0:17,0.0,0.0
...,...,...,...,...,...,...,...,...
3441,3441,Who did the athlete with a silver medal in the...,Law School Admission Council,None,Cara Dunne,Cara Dunne,0.0,0.0
3448,3448,Who created the largest and most popular gener...,MediaWiki,{{\n LLMQA(\n 'Who created the large...,A community of volunteer editors created Wikip...,A community of volunteer editors created Wikip...,0.0,0.0
3451,3451,What is the place of scenic beauty with a type...,Dorohatchō,"SELECT ""site"" FROM w WHERE ""municipality"" = {{...",* dorohatchō 瀞八丁 kitabatake-shi yakata ato teien,dorohatchō 瀞八丁 kitabatake-shi yakata ato teien,0.0,0.0
3461,3461,What is the league whose city has a history of...,Hong Kong Premier League,"SELECT ""league"" FROM w WHERE ""city"" = {{\n ...",Chinese Super League,Chinese Super League,0.0,0.0


In [20]:
common_mistake.to_csv('./common_mistaske.csv', index = False)

Those common mistakes can be attributed to:

1. text normlaization -> reason(id): details
   1.1) numeridcal(4, 3460): 6th -> sixth, 3 -> three.
   1.2) date(3464): 17 August 1997 -> 1997-8-17.
   1.3) unstructed lengthy answer(3, 3462): The address of the museum located in a Victorian House is 503 Peeples Street SW in the West End neighborhood of Atlanta, Georgia -> 503 Peeples Street SW
   
2. wrong inference
   1.1) using only document instead of table information(6, 3461): the billionaire information is only present in document.content, the history information is only present in document.content.
   1.2) blendsql syntax is too detailed on date filtering(7): harry william payne ( (1892-9-51969-7-5,p28061d) ) was.... -> %born 5 is too specific of a sql format, missing 1892-9-5 in the document.
       {{
        LLMQA(
            'What is the time difference between these two athletes?',
            (
                SELECT w."time" AS 'Time', documents.title AS 'Athlete', documents.content FROM documents
                JOIN {{
                    LLMJoin(
                        left_on='w::athlete',
                        right_on='documents::title'
                    )
                }} WHERE w."athlete" = 'josé reliegos' OR documents.content LIKE '%born 5 september 1892%' AND documents.content LIKE '%1928 olympics%'
            )
        )
3. wrong answer in the hybridqa table
   3.1) missing information (3465): Blendsql returned Craig Ferguson, which should be the correct answer because Craig Kilborn has been missing from the document.content and table w.
   3.2) more information(3451): Blendsql returend the full answer that is present in the table "dorohatchō 瀞八丁 kitabatake-shi yakata ato teien", but the hybridqa label the correct answer as "dorohatchō"

   

   
   


In [40]:
check_idx = 3451

print(common_mistake['question'][check_idx])

print(blendsql_preds[blendsql_preds['idx']== check_idx]['dataset_vars'].values[0]['table_id'])

print(common_mistake['blendsql_text'][check_idx])

print(common_mistake['pred_sql'][check_idx])

print(blendsql_preds[blendsql_preds['idx']== check_idx]['input_program_args'].iloc[0]['serialized_db'])

What is the place of scenic beauty with a type number less than 8 , located in the city that had an estimated population of 17,727 ?
List_of_Places_of_Scenic_Beauty_of_Japan_(Mie)_0
* dorohatchō 瀞八丁 kitabatake-shi yakata ato teien
SELECT "site" FROM w WHERE "municipality" = {{
    LLMQA(
        'Which city had an estimated population of 17,727?',
        (
            SELECT documents.title, documents.content FROM documents 
                WHERE documents MATCH 'population'
        ),
        options='w::municipality'
    )
}} AND CAST("ref" AS INTEGER) < 8

CREATE TABLE "w" (
"index" INTEGER,
  "site" TEXT,
  "municipality" TEXT,
  "type" TEXT,
  "ref" TEXT
)
/*
3 example rows:
SELECT * FROM "w" LIMIT 3
 index                                                                            site municipality  type   ref
     0                                * dorohatchō 瀞八丁 kitabatake-shi yakata ato teien       kumano 5 , 6 [ 1 ]
     1                        former moroto family gardens 旧

### 3.3 what type of mistakes does Blendsql make, is it in the parser or the blender?

In [22]:
blendsql_flag = agg_table['blendsql_'+ show_metric] == 0.0
blendsql_mistake = agg_table[blendsql_flag][show_columns]
blendsql_mistake

,idx,question,answer_text,pred_sql,blendsql_text,gpt_text,blendsql_denotation_acc,gpt_denotation_acc
1,1,What was the nickname of the gold medal winner...,Starke Rudolf,{{\n LLMQA(\n 'What was the nickname...,Johan Rudolf Starke,Rudolf Starke,0.0,0.0
3,3,What is that address of the museum located in ...,503 Peeples Street SW,{{\n LLMQA(\n 'What is the address o...,The address of the museum located in a Victori...,The address of the museum located in a Victori...,0.0,0.0
4,4,What place was achieved by the person who fini...,sixth,"SELECT ""position"" FROM w WHERE ""time"" = '2:13....",6th,6th,0.0,0.0
6,6,How many years was the Mexican billionaire Sli...,3,{{\n LLMQA(\n 'How many years was Sl...,4,4 years,0.0,0.0
7,7,What is the difference in time between José Re...,0:06,{{\n LLMQA(\n 'What is the time diff...,0:17,0:17,0.0,0.0
...,...,...,...,...,...,...,...,...
3448,3448,Who created the largest and most popular gener...,MediaWiki,{{\n LLMQA(\n 'Who created the large...,A community of volunteer editors created Wikip...,A community of volunteer editors created Wikip...,0.0,0.0
3449,3449,The position of the youngest athlete is usuall...,defenders and forwards,{{\n LLMQA(\n 'Between which two typ...,Kim Ju-young and Koh Myong-jin,Defenders and Forwards,0.0,1.0
3451,3451,What is the place of scenic beauty with a type...,Dorohatchō,"SELECT ""site"" FROM w WHERE ""municipality"" = {{...",* dorohatchō 瀞八丁 kitabatake-shi yakata ato teien,dorohatchō 瀞八丁 kitabatake-shi yakata ato teien,0.0,0.0
3461,3461,What is the league whose city has a history of...,Hong Kong Premier League,"SELECT ""league"" FROM w WHERE ""city"" = {{\n ...",Chinese Super League,Chinese Super League,0.0,0.0
